In [1]:
import numpy as np
import tqdm
import math

In [2]:
def matrix_pm_fast(B:np.ndarray,n:int, m:int):
    
    D = B.copy()
    D = D[0]
    
    if n == 0:
        return np.identity(B.shape[0])[0]
    if n == 1:
        return D
    
    for x in tqdm.tqdm(range(n-1)):
        D = (D @ B) % m

    return D

In [3]:
def count_walks(n:int=1,MOD:int = 0):
    
    if (n < 1):
        raise ValueError("Walk distance must be 1 or greater!")
            
    states_p1 = []
    for L in 'ABCD':
        for R in 'ABCD':
            states_p1.append(L+R)
    C1 = [[x] for x in states_p1]
    
    C2 = []
    for last in states_p1:
        for new in states_p1:
            LL_match = last[0] == new[0]
            RR_match = last[1] == new[1]
            LR_match = last[0] == new[1]
            RL_match = last[1] == new[0]
            state = (LL_match, RR_match, LR_match, RL_match)

            X0 = (False, False, False, False)
            X1 = (True, True, True, True)
            X2 = (True, False, False, False)
            X3 = (False, True, False, False)
            X4 = (True, True, False, False)
            M = ['AA','BB','CC','DD']
            if state not in [X0, X1, X2, X3, X4]:
                continue
            if state == X1 and last == new:
                continue
            if state == X0 and (last not in M) and (new not in M):
                continue
            C2.append([last, new])
    C3 = []
    for pair in C2:
        last = pair[1]
        for new in states_p1:
            if [last, new] in C2:
                C3.append(pair + [new])
    
    C = C1+C2+C3
    
    if n == 1:
        output = len(C1)
        if MOD not in [0,1]:
            return output%MOD
        else:
            return output
    if n == 2:
        output = len(C2)
        if MOD not in [0,1]:
            return output%MOD
        else:
            return output
    if n == 3:
        output = len(C3)
        if MOD not in [0,1]:
            return output%MOD
        else:
            return output
    else:
        A = np.zeros((len(C3)+1,(len(C3)+1)))
        A[0][1:] = np.ones(len(C3))
        
        B = A.copy()
        
        for c0 in C3:
            idx = C3.index(c0)+1
            edges = []
            for c1 in C1:
                if ([c0[1],c0[2],c1[0]] in C3):
                    if (c0 == [c0[1],c0[2],c1[0]]):
                        continue
                    if (c0[0][0]+c0[1][0]+c0[2][0] == c0[1][0]+c0[2][0]+c1[0][0]):
                        continue
                    if (c0[0][1]+c0[1][1]+c0[2][1] == c0[1][1]+c0[2][1]+c1[0][1]):
                        continue
                    edges.append([c0[1],c0[2],c1[0]])
            
            for edge in edges:
                B[idx][C3.index(edge)+1] += 1
    
    if MOD in [0,1]:
        B = np.linalg.matrix_power(B, n)
        output = np.sum(B[0][1:])
        return int(output%MOD)
    
    # print('USING MOD'
    J = matrix_pm_fast(B,n-2,MOD)
    output = np.sum(J)

    return int(output%MOD)

In [6]:
count_walks(2**24, 3141592653)

  0%|▏                              | 71857/16777213 [00:06<24:08, 11530.76it/s]


KeyboardInterrupt: 